<a href="https://colab.research.google.com/github/outinletter/DataAnalysis/blob/main/%EC%B0%B8%EA%B3%A0%2C_%EC%BD%94%EB%94%A9%2C_SMOTE%2C_%EB%B6%80%EC%A1%B1%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 부족한 데이터 불규형

- 데이터 분석 시 데이터의 불균형의 문제 (예, 부도 예측시 전체 기업의 3% 내외)
- 비대칭 데이터셋에서는 정확도가 높아도 재현율이 현저히 작아짐


# 데이터 불균형 처리 방법
- 언더샘플링
    - 무작위로 정상 데이터를 일부만 선택(EasyEnsemble, BalanceCascade)
    - 언더샘플링의 경우 데이터의 소실이 매우 크고, 중요한 정상데이터를 잃게되는 단점
- 오버샘플링
    - 무작위로 소수 데이터를 복제
    - 정보 손실은 없으나, 복제된 관측치로 오버피팅

# SMOTE(synthetic minority oversampling technique)
- 다수 클래스를 샘플링하고 기존 소수 샘플을 보간하여 새로운 소수 인스턴스를 합성
- 소수데이터들 사이를 보간하여 작동
- 새로운 사례의 데이터 예측엔 취약

In [ ]:
# 데이터 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit_transform(X_train)
X_train = scaler.fit_transform(X_train)

In [ ]:
# 데이터 복제
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# 모델 설정
sm = SMOTE(ratio='auto', kind='regular')

In [ ]:
# train데이터를 넣어 복제
X_resampled, y_resampled = sm.fit_sample(X_train,list(y_train))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))


In [ ]:
# 정상데이터수를 기준으로 50% : 50%로 소수데이터가 합성
X_resampled.shape, y_resampled.shape

# Borderline-SMOTE
## A New Over-Sampling Method in Imbalanced Data Sets Learning
- 연예인 얼굴처럼 데이터가 많이 존재하는 class를 majority class라고 하고, 일반인 얼굴처럼 적게 존재하는 class를 minority class
- 기존의 SMOTE 알고리즘이 단순히 minority class에서 랜덤하게 샘플링 했다면, Borderline-SMOTE는 다른 class와의 경계(borderline)에 있는 샘플들을 늘림으로써 분류하기 더 어려운 부분에 집중

## 샘플링 순서
1. minority class에 속한 모든 example에 대하여 minor, major 구분 없이 nearest neighbor를 추출
2. 뽑아낸 nearest neighbor 중 절반 이상이 majority class이며 이 example을 DANGER 라고 하는데, 이는 곧 borderline에 있는 분류기가 어려워하는 example의 set을 의미
3. 경계에 있는 샘플들, 즉 DANGER set에 대하여 nearest neighbor들을 다시 뽑는다. 이 때는 minority class에서만 뽑아낸다.
4. 인위적으로 생성한 데이터, synthetic example을 만드는데, 위와 같이 기존의 minority example (p'i) 에 뽑아낸 nearest neighbor 과의 차이(dif j)를 랜덤 비율(r j)로 더한 값을 사용

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import BorderlineSMOTE

X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], 
                           n_informative=3, n_redundant=1, flip_y=0, n_features=20, 
                           n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))

In [ ]:
Original dataset shape Counter({1: 900, 0: 100})
sm = BorderlineSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))
Resampled dataset shape Counter({0: 900, 1: 900})

## Imbalanced small dataset으로 변형

In [ ]:
import tensorflow as tf
from imblearn.over_sampling import BorderlineSMOTE
import numpy as np
import random

In [ ]:
EPOCHS = 100

In [ ]:
# 모델
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten() 
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [ ]:
# 불규형 데이터셋
cifar10 = tf.keras.datasets.cifar10 # 32x32x3

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()
for x, y in zip(x_train, y_train):
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:])
        y_train_small.append(y)

x_test_small = list()
y_test_small = list()
for x, y in zip(x_test, y_test):
    if y == 0 or y == 1:
        x_test_small.append(x[:])
        y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

In [1]:
# Keras API 모델 학습 (불균형한 데이터셋)
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)